In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Load cancer data

In [2]:
en = cptac.Endometrial()
br = cptac.Brca()
cl = cptac.Colon()

In [3]:
gene = 'PIK3CA'

# Endometrial

## Filter out hotspot mutations
Hotspots are:
E542K
E545K
H1047R


#### Get the mutation type, and ptorteomics for PIK3CA

In [4]:
mut = en.get_genotype_all_vars(gene, mutation_hotspot=['E542K', 'E545K', 'H1047R'])
prot = en.get_proteomics(tissue_type="tumor")

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)


#### Join mutation type and proteomics together

In [5]:
joined = mut.join(prot)

#### Select samples containing hotspot mutations

In [6]:
en_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]

#### Join the proteomics for the wildtype mutations to the hotspot mutations

In [7]:
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

endo_hotspot = en_hotspot_df.append(wildtype)

#### Drop unncecesary columns
The resulting dataframe is what we will be working with from this point forward. It contains the mutations type and the proteomics for each gene.

In [8]:
prot_and_mutations = endo_hotspot.drop(columns = ["Mutation_Status", "Location"])
prot_and_mutations.head()

Name,Mutation,A1BG,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,AAED1,AAGAB,...,ZSWIM8,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,Missense_Mutation,-1.180,-0.863,-0.802,0.222,0.2560,0.665,1.2800,-0.33900,0.412,...,-0.08770,NaN,0.0229,0.109,NaN,-0.332,-0.4330,-1.020,-0.1230,-0.0859
C3L-00032,Missense_Mutation,-0.528,-1.320,0.435,NaN,-0.2400,1.040,-0.0213,-0.04790,0.419,...,0.00112,-0.1450,0.0105,-0.116,NaN,0.151,-0.0740,-0.540,0.3200,-0.4190
C3L-00362,Missense_Mutation,-0.924,-0.445,1.570,-0.678,0.1730,0.436,0.2270,-0.00248,0.479,...,-0.04960,NaN,0.1080,-0.153,NaN,0.109,-0.3360,-0.822,-0.0338,0.1210
C3L-00601,Missense_Mutation,-0.454,-0.242,NaN,NaN,0.2580,0.219,-0.2490,-1.33000,0.204,...,-0.15800,NaN,0.6670,1.300,0.441,0.130,-0.0659,-0.923,-0.1630,0.1120
C3L-00605,Missense_Mutation,-0.240,0.594,3.400,0.154,0.0932,0.283,-0.0789,-0.61100,0.370,...,-0.05110,-0.0136,0.2140,0.683,0.433,-0.208,-0.3470,-0.911,0.0692,-0.2320


## Run T-test and difference of Median for each cancer type

#### Get the median proteomic expression among all samples, for each gene.
This is done for missense mutations and wildtype separately


In [9]:
missense = prot_and_mutations[prot_and_mutations.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = prot_and_mutations[prot_and_mutations.Mutation == "Wildtype_Tumor"] #get all wildtype mutations
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype

#### Get the medain difference between missense and wildtype for each gene.
This is done by (for each gene) subtracting the median of the missense mutations from the median of the wildtype.
This means that if the difference is positive the proteomics of the wildtype have higher expression than the missense mutation. If the difference is negative then the missense mutation has a higher expression than the wildtype.

In [10]:
endo_d = {}

for gene in prot_and_mutations:
    if gene == "Mutation": continue
    dif_endo = wt_med[gene] - missense_med[gene]
    endo_d[gene] = dif_endo

median_diff = pd.DataFrame.from_dict(endo_d, orient='index', columns=['Difference_In_Median'])
median_diff.head()

,Difference_In_Median
A1BG,-0.055500
A2M,-0.038000
A2ML1,-1.105850
A4GALT,-0.111075
AAAS,-0.156100


#### Do a t-test for every gene.

In [11]:
genes = list(prot_and_mutations.columns[1:])
t_test = u.wrap_ttest(prot_and_mutations, 'Mutation', genes, return_all = True)
t_test = t_test.set_index('Comparison')
t_test.head()

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:153: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals <= alphacBonf
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:257: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


,P_Value
Comparison,
SLC25A14,0.000645
CORO7-PAM16,0.000877
RIPK1,0.001035
LYRM9,0.001181
GAS6,0.001208


#### Join difference in median and t-test p value into the same dataframe

In [12]:
# Step 4 join median and p value together
endo_df = median_diff.join(t_test)
endo_df.head()

,Difference_In_Median,P_Value
A1BG,-0.055500,0.434646
A2M,-0.038000,0.714215
A2ML1,-1.105850,0.037044
A4GALT,-0.111075,0.837021
AAAS,-0.156100,0.664545


# Brca
The Brca data uses different database_IDs, giving us multiple proteomic values for each gene. To deal with this, we will first separate genes that have multiple proteomics values recoreded (multiple database_IDs) from ones that don't.

## Dealing with multiple database_IDs

#### Separate genes with multiple database_IDs from genes with multiple database_IDs

In [13]:
proteomics = br.get_proteomics(tissue_type="tumor")
fil = proteomics.columns.get_level_values('Name').duplicated(False)
duplicates = proteomics[proteomics.columns[fil]]
no_duplicates = proteomics[proteomics.columns[~fil]]
no_duplicates

Name,A1BG,A2M,A2ML1,AAAS,AACS,AADAT,AAED1,AAGAB,AAK1,AAMDC,...,ZSCAN31,ZSWIM8,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
Database_ID,NP_570602.2,NP_000005.2,NP_653271.2|NP_001269353.1,NP_056480.1|NP_001166937.1,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_057312.1|NP_001273611.1,NP_714542.1,NP_078942.3|NP_001258815.1,NP_055726.3,NP_078960.1|NP_001303886.1|NP_001303887.1,...,NP_665916.1|NP_001230172.1|NP_116078.4|NP_001166148.1|NP_001229731.1|NP_001012458.1,NP_055852.2|NP_001229416.1|NP_001229417.1,NP_004715.1,NP_060445.3|NP_001274750.1,NP_008988.2|NP_001005413.1,NP_079388.3|NP_001035743.1,NP_078922.1,NP_001010972.1,NP_055928.3,NP_056349.1|NP_001295166.1
CPT000814,-0.6712,-0.2075,2.7959,1.3969,-1.0899,NaN,1.6708,-0.3484,-0.4756,-0.7299,...,-5.2868,-0.6536,0.3384,2.1169,1.3910,-2.1230,0.9136,-0.8082,-1.4793,0.9136
CPT001846,1.3964,1.3302,-5.0948,0.7674,-1.6845,NaN,2.1022,-0.5814,0.2916,-2.2857,...,-0.7592,0.4711,0.6018,0.2062,-0.2137,-2.1219,0.0860,2.5814,-0.2852,-0.1074
X01BR001,2.0219,1.6269,-3.2943,0.3352,-1.0739,1.2255,0.2754,-1.1187,-0.0534,-0.2519,...,NaN,0.2306,-0.3010,0.3395,-0.5316,NaN,0.4996,0.7622,-1.5607,0.0256
X01BR008,-0.5290,0.3267,1.4342,0.4938,-2.8676,NaN,NaN,-1.0691,-0.3643,-1.8173,...,-2.1789,0.2695,0.1506,1.0498,0.7546,1.7889,-0.2499,-0.2590,-0.1263,0.3725
X01BR009,1.2556,3.4489,2.8043,-0.2956,-1.7261,NaN,NaN,-2.0471,-0.3547,-0.8298,...,-2.3990,-0.2596,0.1898,-0.5010,-0.4189,0.3080,0.5057,0.2181,-0.2288,-0.2750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X21BR001,-0.6610,-0.6402,-4.8578,1.2319,-1.6491,NaN,NaN,-0.3074,-0.3074,-0.0266,...,-0.2528,0.5090,0.0306,0.4908,-0.5570,2.3864,0.3764,-0.6974,1.3541,1.1123
X21BR002,-1.3735,0.4227,-4.9553,0.6327,-3.1434,NaN,NaN,0.3071,0.7562,-1.6912,...,-3.3351,0.1548,1.0792,-0.6619,-1.4444,-0.3704,0.4909,0.3938,0.2992,-0.3494
X21BR010,1.1583,0.3329,-5.7358,-0.1658,-2.0413,-1.2433,0.9090,-0.2410,0.6717,-0.1651,...,-0.7054,0.2752,0.8850,-2.6704,-0.9444,-1.9717,0.0650,0.6300,-0.0686,0.1798


### Dealing with genes that don't have multiple database_IDs
These will be processed the same way the data for endometrial was.

In [14]:
#FOR NO DUPLICATES
#Flatten the multiindex (since there are no duplicate gene names, this won't be a problem)
no_duplicates = br.reduce_multiindex(no_duplicates, flatten=True)

#get mutation_tyoe
mut_type = br.get_genotype_all_vars('PIK3CA')

#join proteomics and mutation type
joined = mut_type.join(no_duplicates)

#select samples containing hotspot mutations
br_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]


#join wildtype proteomics to missense mutation proteomics 
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]
brca_hotspot = br_hotspot_df.append(wildtype)

#drop unncessary columns
prot_and_mutations = brca_hotspot.drop(columns = ["Mutation_Status", "Location"])

# Get the difference in medians
missense = prot_and_mutations[prot_and_mutations.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = prot_and_mutations[prot_and_mutations.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


no_dup_d = {}

for prot in no_duplicates:
    dif_brca = wt_med[prot] - missense_med[prot]
    no_dup_d[prot] = dif_brca

median_diff = pd.DataFrame.from_dict(no_dup_d, orient='index', columns=['Difference_In_Median'])

#Do a t test for every gene and report the p-value
genes = list(prot_and_mutations.columns[1:])
t_test = u.wrap_ttest(prot_and_mutations, 'Mutation', genes, return_all = True)
t_test = t_test.set_index('Comparison')

#join median and p value together
no_dup_df = median_diff.join(t_test)
no_dup_df.head()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 83 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:153: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals <= alphacBonf
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:257: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


,Difference_In_Median,P_Value
A1BG_NP_570602.2,-0.63510,0.830859
A2M_NP_000005.2,-0.05110,0.999049
A2ML1_NP_653271.2|NP_001269353.1,1.26120,0.165194
AAAS_NP_056480.1|NP_001166937.1,-0.13550,0.201707
AACS_NP_076417.2|NP_001306769.1|NP_001306768.1,0.01775,0.652935


## Dealing with genes that do have multiple database_IDs

#### Get t-test results and median difference

In [15]:
#DEALING WITH DUPLICATES
#flatten the multiindex
duplicates = br.reduce_multiindex(duplicates, flatten=True)

# We need to figure which of databaseID we want to use, for each protein
proteomics = br.get_proteomics(tissue_type="tumor")
fil = proteomics.columns.get_level_values('Name').duplicated(False)
duplicates = proteomics[proteomics.columns[fil]]
duplicate_gene_names = list(set(duplicates.columns.get_level_values('Name'))) #get a list of genes names that are duplicated
duplicates = br.reduce_multiindex(duplicates, flatten=True)
duplicates

#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest

mut_type = br.get_genotype_all_vars("PIK3CA")


joined = mut_type.join(duplicates)

#select samples containing hotspot mutations
br_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]


#join wildtype proteomics to missense mutation proteomics 
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]
brca_hotspot = br_hotspot_df.append(wildtype)

#drop unncessary columns
prot_and_mutations = brca_hotspot.drop(columns = ["Mutation_Status", "Location"])


# get the difference in medians
missense = prot_and_mutations[prot_and_mutations.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = prot_and_mutations[prot_and_mutations.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


duplicates_d = {}

for prot in duplicates:
    dif_brca = wt_med[prot] - missense_med[prot]
    duplicates_d[prot] = dif_brca
    

median_diff = pd.DataFrame.from_dict(duplicates_d, orient='index', columns=['Difference_In_Median'])


#step 3 do a t test for every gene and report the p-value
cols = list(prot_and_mutations.columns[1:])
t_test = u.wrap_ttest(prot_and_mutations, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
dup_df = median_diff.join(t_test)
dup_df.head()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 83 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)


,Difference_In_Median,P_Value
ABCB7_NP_001258628.1|NP_001258626.1,0.2932,0.238399
ABCB7_NP_004290.2|NP_001258625.1|NP_001258627.1,0.2573,0.537629
ABLIM3_NP_001287947.1|NP_001287944.1,-0.1830,0.781017
ABLIM3_NP_001287956.1,0.4862,0.166418
ACIN1_NP_001158286.1,-0.1520,0.783389


#### Select the database_ID that gives a lower p value from the t-test, and append selected database_ID to the no_duplicate dataframe

In [16]:
#for each duplicated gene name select the one with the min p val and append to no_duplicat df
for name in duplicate_gene_names:
    gene_duplicate = dup_df[dup_df.index.str.contains(name)]
    #select the one with the min p value for t test
    min_row = gene_duplicate[gene_duplicate.P_Value == gene_duplicate.P_Value.min()]
    #append the selected version onto no_duplicate dataframe
    no_dup_df=no_dup_df.append(min_row)
no_dup_df=no_dup_df.sort_index()
no_dup_df.head()

,Difference_In_Median,P_Value
A1BG_NP_570602.2,-0.63510,0.830859
A2ML1_NP_653271.2|NP_001269353.1,1.26120,0.165194
A2M_NP_000005.2,-0.05110,0.999049
AAAS_NP_056480.1|NP_001166937.1,-0.13550,0.201707
AACS_NP_076417.2|NP_001306769.1|NP_001306768.1,0.01775,0.652935


#### Reformat the table by getting rid of databaseIDs so we can join it to Endo and Brca

In [18]:
split = no_dup_df.index.str.split('_', 1).str[0]
no_dup_df['new_index']=split
brca_df = no_dup_df.set_index("new_index")
brca_df.head()

,Difference_In_Median,P_Value
new_index,,
A1BG,-0.63510,0.830859
A2ML1,1.26120,0.165194
A2M,-0.05110,0.999049
AAAS,-0.13550,0.201707
AACS,0.01775,0.652935


# Colon

In [20]:
#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest
proteomics = cl.get_proteomics(tissue_type="tumor")

prot_list = list(proteomics.columns)

mut_type = cl.get_genotype_all_vars("PIK3CA")


joined = mut_type.join(proteomics)

#select samples containing hotspot mutations
cl_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]

#join wildtype proteomics to missense mutation proteomics 
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]
colon_hotspot = cl_hotspot_df.append(wildtype)

#drop unncessary columns
prot_and_mutations = colon_hotspot.drop(columns = ["Mutation_Status", "Location"])


#get the difference in medians
missense = prot_and_mutations[prot_and_mutations.Mutation == "nonsynonymous SNV"] #get all missense_mutations
wt = prot_and_mutations[prot_and_mutations.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


colon_d = {}

for prot in proteomics:
    dif_colon = wt_med[prot] - missense_med[prot]
    colon_d[prot] = dif_colon

median_diff = pd.DataFrame.from_dict(colon_d, orient='index', columns=['Difference_In_Median'])

#do a t test for every gene and report the p-value
cols = list(prot_and_mutations.columns[1:])
t_test = u.wrap_ttest(prot_and_mutations, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

#join median and p value together
colon_df = median_diff.join(t_test)
colon_df.head()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:153: RuntimeWarning: invalid value encountered in less_equal
  rejec

,Difference_In_Median,P_Value
A1BG,0.61600,0.263957
A1CF,0.03225,0.913020
A2M,0.26800,0.265891
AAAS,0.26315,0.070194
AACS,0.00850,0.113686


# Combine Endo, Brca, and Colon into one table

#### Combine brca and endo

In [21]:
brca_df=brca_df.rename_axis(None)
brca_endo = brca_df.join(endo_df, lsuffix='_Brca', rsuffix='_Endo')
brca_endo.head()

,Difference_In_Median_Brca,P_Value_Brca,Difference_In_Median_Endo,P_Value_Endo
A1BG,-0.63510,0.830859,-0.05550,0.434646
A2M,-0.05110,0.999049,-0.03800,0.714215
A2ML1,1.26120,0.165194,-1.10585,0.037044
AAAS,-0.13550,0.201707,-0.15610,0.664545
AACS,0.01775,0.652935,-0.02150,0.732714


#### Combine colon

In [22]:
final_table = brca_endo.join(colon_df, rsuffix='_colon')
final_table.head()

,Difference_In_Median_Brca,P_Value_Brca,Difference_In_Median_Endo,P_Value_Endo,Difference_In_Median,P_Value
A1BG,-0.63510,0.830859,-0.05550,0.434646,0.61600,0.263957
A2M,-0.05110,0.999049,-0.03800,0.714215,0.26800,0.265891
A2ML1,1.26120,0.165194,-1.10585,0.037044,NaN,NaN
AAAS,-0.13550,0.201707,-0.15610,0.664545,0.26315,0.070194
AACS,0.01775,0.652935,-0.02150,0.732714,0.00850,0.113686


#### Only rows that contain a significant pvalue

In [23]:
significant = final_table.loc[(final_table['P_Value_Brca'] <= .05) |
                (final_table['P_Value_Endo'] <= .05) |
                (final_table['P_Value'] <= .05)]
significant.head()

,Difference_In_Median_Brca,P_Value_Brca,Difference_In_Median_Endo,P_Value_Endo,Difference_In_Median,P_Value
A2ML1,1.26120,0.165194,-1.10585,0.037044,NaN,NaN
AADAT,1.33200,0.027477,0.18535,0.690613,NaN,NaN
AAGAB,0.09455,0.909622,-0.24350,0.139632,-0.1390,0.039189
AASDHPPT,-0.51445,0.003620,0.10590,0.428884,0.0628,0.973515
AATF,0.16905,0.042271,0.18170,0.915482,-0.0250,0.568821


In [28]:
# significant.to_csv("~/WhenMutationsDontMatter/PIK3CA/csv_files/proteomics.csv")